# Introduction to Artificial Intelligence
Laboratory 1



Exercise 1 Route searching 
1.	Create a set of cities (as points) with coordinates x, y on a plane with height as z coordinate. The cost of going from city A to city B is equal to the Euclidean distance between two cities, if there exists a road. You should define scenarios according to two criteria: 
a.	There are all the direct connections / c.a. 80% of possible connections
b.	The problem is symmetrical / asymmetrical (in asymmetrical – going up is height +10%, going down: -10%)
You should choose the coordinates randomly from the range <-100, 100> for x,y and <0, 50> for z.
2.	Represent the created map as a weighted (directed) graph, where cities are the nodes and roads are the edges of the graph.
3.	In the created scene, solve the traveling salesman problem: The salesman starts from a chosen city and has to visit every city exactly once before returning to the starting city. The goal is to find a path with the lowest cost.
In the problem, we define state as a partial or full path from the starting city and the corresponding state. You should represent the search problem in a form of state tree.
a.	Implement a full search of the tree, using BFS and DFS methods.
b.	Approximate the solution using greedy search (NN and Dijkstra)
c.	Solve/approximate the solution using A* with inadmissible/admissible heuristics
d.	Approximate the solution using ACO algorithm
4.	Test each algorithm, in each scenario, for n=5…20 cities, in terms of the found path cost, time and memory consumption.


In [2]:
def euclidean_distance(a,b)->float:  
    """ 
    points as a 
    tuple or list for example a = (1,2,3) , b = (4,5,6)
    """
    import math
    dimension = len(a)
    if (dimension !=len(b)):
        print("Dimension missmatch")
        return 0 
    dist = 0
    for i in range(dimension):
        dist += math.pow(a[i]-b[i],2)
    return math.sqrt(dist)
    
    
    
def euclidean_distance_assymetrical(a,b)->float:  
    """ 
    points as a 
    tuple or list for example a = (1,2,3) , b = (4,5,6)
    """
    import math
    dimension = len(a)
    if (dimension !=len(b)):
        print("Dimension missmatch")
        return 0 
    dist = 0
    for i in range(dimension-1): # let's assume last dimesion as high 
        dist += math.pow(a[i]-b[i],2)
        
    if ((a[-1]-b[-1]) > 0):   # a is higher than b so distance from a to b 
        dist += math.pow()
    return math.sqrt(dist)
    

In [3]:
def name_city(n):
    """Name cities in Excel convention starting from A after Z going to AA then AAA and so on 
            Then calculating distances.
    """
    name = ""
    while True:
        
        rest = n % 26  # getting so called rest and then convert to letter
        name = chr(65+rest) + name # add to already mapped values
        
        
        n  = n // 26 - 1 # move on to 'next' char -1 is neccessary because we have some problem with index without it 26 is mapped to 0->A at last position but 26 // 26 -> 1 at first 

        if (n < 0 ):        # will be fullfiled when we iterate through last chat -> n // 26 - 1  = 0 -1 = -1 
            break
    return name
        
    


In [4]:
euclidean_distance((0,0,0),(0,0,4))

4.0

In [5]:
class CitiesMap():
    def __init__(self,n):
        """
        initialize with number of cities to include in graph 
        """    
        import random
        self.cities = []
        self.routes = dict()
        for i in range(n):
            self.cities.append((random.randint(-101,101),random.randint(-101,101),random.randint(-51,51)))

        # now let's make sure that we don't have duplicated cities so we'll get derired number of unique cities
        
        len_check = len(set(self.cities))
        
        while (len_check != n):
            self.cities.append((random.randint(-101,101),random.randint(-101,101),random.randint(-51,51)))
            len_check = len(set(self.cities))
        self.cities = set(self.cities)
        

        
    def create_routes_symmetrical(self, connections = 1):
        """Creates basic graph symmetrical . Default all connections, however setting connections to 0.5 will result in approx 1/2 connections possible connections 
        """
        # for now example with connections to every city
        cities = list(self.cities)
        indexes = len(cities)
                   
                
        if (connections==1):   
            for i in range(indexes):
                for j in range(i+1 , indexes):  # to not double calcualte
                    name_i = name_city(i)
                    name_j = name_city(j)
                    distance = euclidean_distance(cities[i], cities[j])
                
                    # Store the route only once, as it is undirected
                    self.routes[(name_i, name_j)] = distance
            return self.routes
        else:
            import random
            # calculate number of citeis from which to write connections 
            cons =  indexes * connections
            # random sampling 
            index_cons = random.sample(range(0,indexes),int(cons))
            
            # same as previous but now first loop only iterate through previously generated indexes
            # however in this method we'll be getting quite some variatons because sampling low indexes will result in getting more connections then sampling from high indexes
            
            for i in index_cons:
                for j in range(i+1 , indexes):  # to not double calcualte
                    name_i = name_city(i)
                    name_j = name_city(j)
                    distance = euclidean_distance(cities[i], cities[j])
                
                    # Store the route only once, as it is undirected
                    self.routes[(name_i, name_j)] = distance
            return self.routes
            
            
    def create_routes_asymmetrical(self, connections = 1):
        """Creates assymetrical graph.
            Now he have to consider that going from city located 'lower' to 'higher' and vice versa is not the same
        Default all connections, however setting connections to 0.5 will result in approx 1/2 connections possible connections 
        """        
        
        
        

In [10]:
test = CitiesMap(20)
test.create_routes_symmetrical(connections=0.8)

{('Q', 'R'): 169.38417871808454,
 ('Q', 'S'): 131.35067567393781,
 ('Q', 'T'): 140.164189435105,
 ('P', 'Q'): 192.88597668052492,
 ('P', 'R'): 161.94443491518936,
 ('P', 'S'): 193.06475597581243,
 ('P', 'T'): 60.340699366182356,
 ('C', 'D'): 120.93800064495858,
 ('C', 'E'): 98.52918349402881,
 ('C', 'F'): 116.81609478149832,
 ('C', 'G'): 112.44998888394787,
 ('C', 'H'): 53.44155686354955,
 ('C', 'I'): 94.2019108086455,
 ('C', 'J'): 103.85566907973777,
 ('C', 'K'): 131.08012816594282,
 ('C', 'L'): 33.85262175962151,
 ('C', 'M'): 158.905632373431,
 ('C', 'N'): 117.57550765359255,
 ('C', 'O'): 58.60034129593445,
 ('C', 'P'): 128.17956155331473,
 ('C', 'Q'): 158.27507700203466,
 ('C', 'R'): 44.27188724235731,
 ('C', 'S'): 76.90253571892151,
 ('C', 'T'): 115.57248807566617,
 ('A', 'B'): 103.77861051295686,
 ('A', 'C'): 119.59096955874219,
 ('A', 'D'): 69.2098258919931,
 ('A', 'E'): 162.63455967290594,
 ('A', 'F'): 123.10970717209915,
 ('A', 'G'): 49.58830507286975,
 ('A', 'H'): 72.097156670